In [1]:
import pandas as pd
from sympy import sequence
from sympy.printing.pytorch import torch
from sympy.printing.tensorflow import tensorflow

books = pd.read_csv("books_cleaned.csv")

In [2]:
books["categories"].value_counts().reset_index()

categories  count
0                                Fiction   2125
1                       Juvenile Fiction    391
2              Biography & Autobiography    312
3                                History    208
4                     Literary Criticism    124
..                                   ...    ...
475                               Clergy      1
476  Aubrey, Jack (Fictitious character)      1
477                                China      1
478                          Black Death      1
479                        Human cloning      1

[480 rows x 2 columns]

## Classifying Book Descriptions
* Kategorileri çok daha az sayıda gruba ayırmak için ***text classification*** tekniğini kullanacağız.
* **Zero-Shot Classification:** Burada önceden eğitilmiş bir LLM'i alarak, daha fazla eğitime ihtiyaç duymadan, modelin metni bir kategoriye atamasını sağlamak için modele biz bir prompt veriyoruz. Burada, prompt'ın bir parçası olarak sınıflandırmak istediğimiz kategorilerimizi de sağlayabiliriz.
    * **Prompt:** Classify the following text as fiction or non-fiction.

In [3]:
books["categories"].value_counts().reset_index().query("count > 50")

categories  count
0                     Fiction   2125
1            Juvenile Fiction    391
2   Biography & Autobiography    312
3                     History    208
4          Literary Criticism    124
5                  Philosophy    117
6                    Religion    117
7     Comics & Graphic Novels    116
8                       Drama     91
9         Juvenile Nonfiction     57
10                    Science     56
11                     Poetry     53

In [4]:
books[books["categories"] == "Juvenile Fiction"]

isbn13      isbn10  \
30    9780006646006  000664600X   
79    9780020442608  0020442602   
85    9780030547744  0030547741   
86    9780060000141  0060000147   
87    9780060001537  0060001534   
...             ...         ...   
4962  9781596792500  1596792507   
4974  9781599900056  159990005X   
5042  9781844580514  1844580512   
5189  9781932416374  1932416374   
5221  9787506435673  7506435675   

                                                  title  \
30                                   Ocean Star Express   
79                       The voyage of the Dawn Treader   
85                             Where the Red Fern Grows   
86                                       Poppy's Return   
87                                    Diary of a Spider   
...                                                 ...   
4962  Sherlock Holmes and the Case of the Hound of t...   
4974                                    The Drift House   
5042                               Attack of the Jaguar   
5189                The Very Persistent Gappers of Frip   
5221                                         Good Fight   

                                  authors        categories  \
30               Mark Haddon;Peter Sutton  Juvenile Fiction   
79                    Clive Staples Lewis  Juvenile Fiction   
85                           Wilson Rawls  Juvenile Fiction   
86                                    Avi  Juvenile Fiction   
87                          Doreen Cronin  Juvenile Fiction   
...                                   ...               ...   
4962  Arthur Conan Doyle;Malvina G. Vogel  Juvenile Fiction   
4974                            Dale Peck  Juvenile Fiction   
5042                         M. A. Harvey  Juvenile Fiction   
5189           George Saunders;Lane Smith  Juvenile Fiction   
5221            Judd Winick;Matthew Clark  Juvenile Fiction   

                                              thumbnail  \
30    http://books.google.com/books/content?id=I2QZA...   
79    http://books.google.com/books/content?id=fDD3C...   
85    http://books.google.com/books/content?id=IHpRw...   
86    http://books.google.com/books/content?id=XbcMJ...   
87    http://books.google.com/books/content?id=UWvZo...   
...                                                 ...   
4962  http://books.google.com/books/content?id=EWgWP...   
4974  http://books.google.com/books/content?id=kbwPY...   
5042  http://books.google.com/books/content?id=3HUdt...   
5189  http://books.google.com/books/content?id=xYJOP...   
5221  http://books.google.com/books/content?id=xGPZA...   

                                            description  published_year  \
30    Joe and his parents are enjoying a summer holi...          2002.0   
79    The "Dawn Treader" is the first ship Narnia ha...          1970.0   
85    A young boy living in the Ozarks achieves his ...          2000.0   
86    There's trouble at Gray House, the girlhood ho...          2006.0   
87    This is the diary ... of a spider. But don't b...          2005.0   
...                                                 ...             ...   
4962  Sherlock Holmes and Dr. Watson travel to the b...          2005.0   
4974  Sent to stay with their uncle in a ship-like h...          2006.0   
5042  This training manual for operatives of Xtreme ...          2004.0   
5189  A novel set in the seaside village of Frip fol...          2006.0   
5221  Presents the adventures of a new group of hero...          2007.0   

      average_rating  num_pages  ratings_count  \
30              3.50       32.0            1.0   
79              4.09      216.0         2869.0   
85              4.37      288.0           95.0   
86              3.99      256.0         1086.0   
87              4.25       40.0         7903.0   
...              ...        ...            ...   
4962            4.51      237.0           28.0   
4974            3.64      437.0          595.0   
5042            3.40      125.0            4.0   
5189            

In [5]:
books[books["categories"] == "Juvenile Nonfiction"]

isbn13      isbn10  \
107   9780060277406  0060277408   
108   9780060278427  0060278420   
228   9780060782139  0060782137   
267   9780060882600  0060882603   
444   9780064462044  0064462048   
445   9780064462341  006446234X   
449   9780066236179  0066236177   
823   9780142302279  0142302279   
832   9780142407226  0142407224   
834   9780142407929  0142407925   
1025  9780198320272  0198320272   
1085  9780237525378  0237525372   
1752  9780375812934  0375812938   
1755  9780375823787  0375823786   
1783  9780380732548  0380732548   
1986  9780395776087  0395776082   
2119  9780439314558  0439314550   
2131  9780439420068  0439420067   
2153  9780439736466  0439736463   
2197  9780440229353  0440229359   
2225  9780440419174  0440419174   
2334  9780448437439  0448437430   
2337  9780448444826  0448444828   
2517  9780486276809  0486276805   
2523  9780486296388  0486296385   
2623  9780521618748  0521618746   
2629  9780525469780  0525469788   
2874  9780590032490  0590032496   
2875  9780590103879  0590103873   
2913  9780618111206  0618111204   
3150  9780679890522  0679890521   
3219  9780689833762  0689833768   
3220  9780689834196  0689834195   
3221  9780689835612  0689835612   
3236  9780689869907  0689869908   
3240  9780689876707  068987670X   
3280  9780723247791  072324779X   
3287  9780736805995  0736805990   
3309  9780739844328  0739844326   
3541  9780753459645  0753459647   
3571  9780756614195  0756614198   
3584  9780761417101  0761417109   
3816  9780789458209  0789458209   
3817  9780791070482  0791070484   
3818  9780791081266  0791081265   
3819  9780792241737  0792241738   
3880  9780805069853  0805069852   
3882  9780805071061  0805071067   
3940  9780810935204  0810935201   
4088  9780822560562  0822560569   
4091  9780823410781  0823410781   
4434  9781400307845  1400307848   
4628  9781560067849  1560067845   
4776  9781580495868  1580495869   
4840  9781586853334  1586853333   
5073  9781857232097  1857232097   
5174  9781931414166  1931414165   

                                                  title  \
107                          The Secret Garden Cookbook   
108                                       Laura's Album   
228                         Time For Kids: Butterflies!   
267                       The Annotated Charlotte's Web   
444                         My Little House Crafts Book   
445                                        Pioneer Girl   
449                    A Light in the Attic Book and CD   
823                                        Dirty Beasts   
832                      The Tough Guide to Fantasyland   
834                                             The BFG   
1025                                      Julius Caesar   
1085                                       Oliver Twist   
1752       Jurassic Park Institute Dinosaur Field Guide   
1755                    Ancient Greece and the Olympics   
1783                                         Anne Frank   
1986                                 An American Plague   
2119                    Conversations with J.K. Rowling   
2131                                               Eat!   
2153                          Where We Are, what We See   
2197                    How Angel Peterson Got His Name   
2225                                   Vincent Van Gogh   
2334                           Who Was John F. Kennedy?   
2337                                Who Was Anne Frank?   
2517                    The Secret Garden Coloring Book   
2523                         Ballet Class Coloring Book   
2623                                             Hamlet   
2629                Boy, Were We Wrong about Dinosaurs!   
2874                                        The witches   
2875                                 Expository Writing   
2913                                        Once a Wolf   
3150                                 The Trail of Tears   
3219              Sad Underwear and Other Complications   
3220                               

In [6]:
category_mapping = { 'Fiction': "Fiction",
                     'Juvenile Fiction': "Children's Fiction",
                     'Biyography & Autobiography': "Nonfiction",
                     'History': "Nonfiction",
                     'Literary Criticism': "Nonfiction",
                     'Philosophy': "Nonfiction",
                     'Religion': "Nonfiction",
                     'Comics & Graphic Novels': "Fiction",
                     'Drama': "Fiction",
                     'Juvenile Nonfiction': "Children's Nonfiction",
                     'Science':"Nonfiction",
                     'Poetry': "Fiction"

}

books["simple_categories"] = books["categories"].map(category_mapping)

In [7]:
books

isbn13      isbn10  \
0     9780002005883  0002005883   
1     9780002261982  0002261987   
2     9780006178736  0006178731   
3     9780006280897  0006280897   
4     9780006280934  0006280935   
...             ...         ...   
5225  9788172235222  8172235224   
5226  9788173031014  8173031010   
5227  9788179921623  817992162X   
5228  9788185300535  8185300534   
5229  9789027712059  9027712050   

                                                  title  \
0                                                Gilead   
1                                          Spider's Web   
2                                        Rage of angels   
3                                        The Four Loves   
4                                   The Problem of Pain   
...                                                 ...   
5225                                  Mistaken Identity   
5226                                Journey to the East   
5227  The Monk Who Sold His Ferrari: A Fable About F...   
5228                                          I Am that   
5229                           The Berlin Phenomenology   

                                           authors  \
0                               Marilynne Robinson   
1                  Charles Osborne;Agatha Christie   
2                                   Sidney Sheldon   
3                              Clive Staples Lewis   
4                              Clive Staples Lewis   
...                                            ...   
5225                              Nayantara Sahgal   
5226                                 Hermann Hesse   
5227                                  Robin Sharma   
5228  Sri Nisargadatta Maharaj;Sudhakar S. Dikshit   
5229                 Georg Wilhelm Friedrich Hegel   

                         categories  \
0                           Fiction   
1     Detective and mystery stories   
2                           Fiction   
3                    Christian life   
4                    Christian life   
...                             ...   
5225        Indic fiction (English)   
5226              Adventure stories   
5227               Health & Fitness   
5228                     Philosophy   
5229                        History   

                                              thumbnail  \
0     http://books.google.com/books/content?id=KQZCP...   
1     http://books.google.com/books/content?id=gA5GP...   
2     http://books.google.com/books/content?id=FKo2T...   
3     http://books.google.com/books/content?id=XhQ5X...   
4     http://books.google.com/books/content?id=Kk-uV...   
...                                                 ...   
5225  http://books.google.com/books/content?id=q-tKP...   
5226  http://books.google.com/books/content?id=rq6JP...   
5227  http://books.google.com/books/content?id=c_7mf...   
5228  http://books.google.com/books/content?id=Fv_JP...   
5229  http://books.google.com/books/content?id=Vy7Sk...   

                                            description  published_year  \
0     A NOVEL THAT READERS and critics have been eag...          2004.0   
1     A new 'Christie for Christmas' -- a full-lengt...          2000.0   
2     A memorable, mesmerizing heroine Jennifer -- b...          1993.0   
3     Lewis' work on the nature of love divides love...          2002.0   
4     "In The Problem of Pain, C.S. Lewis, one of th...          2002.0   
...                                                 ...             ...   
5225  On A Train Journey Home To North India After L...          2003.0   
5226  This book tells the tale of a man who goes on ...          2002.0   
5227  Wisdom to Create a Life of Passion, Purpose, a...          2003.0   
5228  This collection of the timeless teachings of o...          1999.0   
5229  Since the three volume edition ofHegel's Philo...          1981.0   

      average_rating  num_pages  ratings_count  \
0               3.85      247.0          361.0   
1               3.83      241.0         5164.0   
2    

In [8]:
books[~(books["simple_categories"].isna())]

isbn13      isbn10                     title  \
0     9780002005883  0002005883                    Gilead   
2     9780006178736  0006178731            Rage of angels   
8     9780006482079  0006482074       Warhost of Vastmark   
30    9780006646006  000664600X        Ocean Star Express   
46    9780007121014  0007121016        Taken at the Flood   
...             ...         ...                       ...   
5220  9784770028969  4770028962        Coin Locker Babies   
5221  9787506435673  7506435675                Good Fight   
5222  9788122200850  8122200850          Cry, the Peacock   
5228  9788185300535  8185300534                 I Am that   
5229  9789027712059  9027712050  The Berlin Phenomenology   

                                           authors        categories  \
0                               Marilynne Robinson           Fiction   
2                                   Sidney Sheldon           Fiction   
8                                      Janny Wurts           Fiction   
30                        Mark Haddon;Peter Sutton  Juvenile Fiction   
46                                 Agatha Christie           Fiction   
...                                            ...               ...   
5220                                           村上龍           Fiction   
5221                     Judd Winick;Matthew Clark  Juvenile Fiction   
5222                                   Anita Desai           Fiction   
5228  Sri Nisargadatta Maharaj;Sudhakar S. Dikshit        Philosophy   
5229                 Georg Wilhelm Friedrich Hegel           History   

                                              thumbnail  \
0     http://books.google.com/books/content?id=KQZCP...   
2     http://books.google.com/books/content?id=FKo2T...   
8     http://books.google.com/books/content?id=uOL0f...   
30    http://books.google.com/books/content?id=I2QZA...   
46    http://books.google.com/books/content?id=3gWlx...   
...                                                 ...   
5220  http://books.google.com/books/content?id=87DJw...   
5221  http://books.google.com/books/content?id=xGPZA...   
5222  http://books.google.com/books/content?id=_QKwV...   
5228  http://books.google.com/books/content?id=Fv_JP...   
5229  http://books.google.com/books/content?id=Vy7Sk...   

                                            description  published_year  \
0     A NOVEL THAT READERS and critics have been eag...          2004.0   
2     A memorable, mesmerizing heroine Jennifer -- b...          1993.0   
8     Tricked once more by his wily half-brother, Ly...          1995.0   
30    Joe and his parents are enjoying a summer holi...          2002.0   
46    A Few Weeks After Marrying An Attractive Young...          2002.0   
...                                                 ...             ...   
5220  Rescued from the lockers in which they were le...          2002.0   
5221  Presents the adventures of a new group of hero...          2007.0   
5222  This book is the story of a young girl obsesse...          1980.0   
5228  This collection of the timeless teachings of o...          1999.0   
5229  Since the three volume edition ofHegel's Philo...          1981.0   

      average_rating  num_pages  ratings_count  \
0               3.85      247.0          361.0   
2               3.93      512.0        29532.0   
8               4.03      522.0         2966.0   
30              3.50       32.0            1.0   
46              3.71      352.0         8852.0   
...              ...        ...            ...   
5220            3.75      393.0         5560.0   
5221             NaN        NaN            NaN   
5222            3.22      218.0          134.0   
5228            4.51      531.0          104.0   
5229            0.00      210.0            0.0   

                                 title_and_subtitle  \
0                                            Gilead   
2                                    Rage of angels   
8                               Warhost

In [9]:
import torch
from transformers import pipeline

# Model kartına baktığımızda, bu modeli zero-shot classification'da kullanmak istiyorsak, modelin sınıflandırma yapabilmesini istediğimiz
# kategorileri ayarlamamız gerektiğini söyler.
fiction_categories = ["Fiction", "Nonfiction"]

pipe = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")


Device set to use cpu


In [10]:
sequence = books.loc[books["simple_categories"]=="Fiction", "description"].reset_index(drop=True)[0]

In [11]:
pipe(sequence, fiction_categories)

{'sequence': 'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst

In [12]:
import numpy as np

#highest probability
max_index = np.argmax(pipe(sequence, fiction_categories)["scores"])
max_label = pipe(sequence, fiction_categories)["labels"][max_index]
max_label

'Fiction'

In [13]:
def generate_predictions(sequence, categories):
    predictions = pipe(sequence, categories)
    max_index = np.argmax(predictions["scores"])
    max_label = predictions["labels"][max_index]
    return max_label

In [14]:
# tqdm: Döngüde ne kadar zamanın kaldığını bilmemizi sağlıyacaktır ve bunları belli sayıda bir örnekte yapmak biraz zaman alacaktır.
from tqdm import tqdm

actual_cats = []
predicted_cats = []

for i in tqdm(range(0,300)):
    sequence = books.loc[books["simple_categories"]=="Fiction", "description"].reset_index(drop=True)[i]
    predicted_cats += [generate_predictions(sequence, fiction_categories)]
    actual_cats += ["Fiction"]

for i in tqdm(range(0,300)):
    sequence = books.loc[books["simple_categories"]=="Nonfiction", "description"].reset_index(drop=True)[i]
    predicted_cats += [generate_predictions(sequence, fiction_categories)]
    actual_cats += ["Nonfiction"]


100%|██████████| 300/300 [23:38<00:00,  4.73s/it]


In [15]:

prediction_df = pd.DataFrame({"actual_categories": actual_cats, "predicted_categories": predicted_cats})

In [16]:
prediction_df["correct_prediction"] = (
    np.where(prediction_df["actual_categories"] == prediction_df["predicted_categories"], 1, 0)
)

In [17]:
prediction_df["correct_prediction"].sum()/len(prediction_df)

np.float64(0.7816666666666666)

In [18]:
isbns = []
predicted_cats = []

missing_cats = books.loc[books["simple_categories"].isna(), ["isbn13", "description"]].reset_index(drop=True)

In [19]:
for i in tqdm(range(0, len(missing_cats))):
    sequence = missing_cats["description"][i]
    predicted_cats += [generate_predictions(sequence, fiction_categories)]
    isbns += [missing_cats["isbn13"][i]]

100%|██████████| 1775/1775 [1:39:43<00:00,  3.37s/it]  


In [23]:
missing_predicted_df = pd.DataFrame({"isbn13": isbns, "predicted_categories": predicted_cats})

In [24]:
missing_predicted_df

isbn13 predicted_categories
0     9780002261982              Fiction
1     9780006280897           Nonfiction
2     9780006280934           Nonfiction
3     9780006380832           Nonfiction
4     9780006470229              Fiction
...             ...                  ...
1770  9788125026600           Nonfiction
1771  9788171565641              Fiction
1772  9788172235222              Fiction
1773  9788173031014           Nonfiction
1774  9788179921623              Fiction

[1775 rows x 2 columns]

In [25]:
# Son kısımda, eksik olanları değiştirmek için bu kategorileri kullanmak ve orijinal df ile birleştirmek
# Left join yapmamızın nedeni, bu eksik kategorilerin, orijinal df'in bir alt kümesi olması ve tüm gözlemleri tutmak istememizdir.
books = pd.merge(books, missing_predicted_df, on="isbn13", how="left")
# Orijinal simple kategoriler eksik olduğunda, lütfen tahmin edilen kategorilerden gelen değerleri kullan, aksi takdirde ise orijinal değerleri kullan.
books["simple_categories"] = np.where(books["simple_categories"].isna(), books["predicted_categories"], books["simple_categories"])
books = books.drop(columns= ["predicted_categories"])

In [26]:
books

isbn13      isbn10  \
0     9780002005883  0002005883   
1     9780002261982  0002261987   
2     9780006178736  0006178731   
3     9780006280897  0006280897   
4     9780006280934  0006280935   
...             ...         ...   
5225  9788172235222  8172235224   
5226  9788173031014  8173031010   
5227  9788179921623  817992162X   
5228  9788185300535  8185300534   
5229  9789027712059  9027712050   

                                                  title  \
0                                                Gilead   
1                                          Spider's Web   
2                                        Rage of angels   
3                                        The Four Loves   
4                                   The Problem of Pain   
...                                                 ...   
5225                                  Mistaken Identity   
5226                                Journey to the East   
5227  The Monk Who Sold His Ferrari: A Fable About F...   
5228                                          I Am that   
5229                           The Berlin Phenomenology   

                                           authors  \
0                               Marilynne Robinson   
1                  Charles Osborne;Agatha Christie   
2                                   Sidney Sheldon   
3                              Clive Staples Lewis   
4                              Clive Staples Lewis   
...                                            ...   
5225                              Nayantara Sahgal   
5226                                 Hermann Hesse   
5227                                  Robin Sharma   
5228  Sri Nisargadatta Maharaj;Sudhakar S. Dikshit   
5229                 Georg Wilhelm Friedrich Hegel   

                         categories  \
0                           Fiction   
1     Detective and mystery stories   
2                           Fiction   
3                    Christian life   
4                    Christian life   
...                             ...   
5225        Indic fiction (English)   
5226              Adventure stories   
5227               Health & Fitness   
5228                     Philosophy   
5229                        History   

                                              thumbnail  \
0     http://books.google.com/books/content?id=KQZCP...   
1     http://books.google.com/books/content?id=gA5GP...   
2     http://books.google.com/books/content?id=FKo2T...   
3     http://books.google.com/books/content?id=XhQ5X...   
4     http://books.google.com/books/content?id=Kk-uV...   
...                                                 ...   
5225  http://books.google.com/books/content?id=q-tKP...   
5226  http://books.google.com/books/content?id=rq6JP...   
5227  http://books.google.com/books/content?id=c_7mf...   
5228  http://books.google.com/books/content?id=Fv_JP...   
5229  http://books.google.com/books/content?id=Vy7Sk...   

                                            description  published_year  \
0     A NOVEL THAT READERS and critics have been eag...          2004.0   
1     A new 'Christie for Christmas' -- a full-lengt...          2000.0   
2     A memorable, mesmerizing heroine Jennifer -- b...          1993.0   
3     Lewis' work on the nature of love divides love...          2002.0   
4     "In The Problem of Pain, C.S. Lewis, one of th...          2002.0   
...                                                 ...             ...   
5225  On A Train Journey Home To North India After L...          2003.0   
5226  This book tells the tale of a man who goes on ...          2002.0   
5227  Wisdom to Create a Life of Passion, Purpose, a...          2003.0   
5228  This collection of the timeless teachings of o...          1999.0   
5229  Since the three volume edition ofHegel's Philo...          1981.0   

      average_rating  num_pages  ratings_count  \
0               3.85      247.0          361.0   
1               3.83      241.0         5164.0   
2    

In [27]:
books[books["categories"].str.lower().isin(
    [
        "romance",
        "science fiction",
        "scifi",
        "fantasy",
        "horror",
        "mystery",
        "thriller",
        "comedy",
        "crime",
        "historical"
    ])
]

isbn13      isbn10                                     title  \
24    9780006513087  0006513085                                   Gravity   
486   9780099410355  0099410354                                   Traitor   
489   9780099422341  0099422344                            Yeats is Dead!   
502   9780099446729  0099446723                            Blackwood Farm   
1101  9780261102422  0261102427                          The Silmarillion   
1446  9780340837955  0340837950                Stranger in a Strange Land   
1450  9780345251220  0345251229                      Visions from Nowhere   
2861  9780575075597  0575075597                                    Replay   
2876  9780590254762  0590254766      The lion, the witch and the wardrobe   
3305  9780739423851  0739423851                           Wizard's Castle   
3306  9780739439708  0739439707                              Time Quartet   
3582  9780760728505  076072850X              20,000 Leagues Under the Sea   
4508  9781416502043  1416502041  The Voyage of the Jerle Shannara Trilogy   
5011  9781841494081  1841494089            The Darkness that Comes Before   
5072  9781857231359  185723135X                            Use of Weapons   
5089  9781857989342  1857989341                       The Gods Themselves   

                        authors       categories  \
24               Tess Gerritsen  Science fiction   
486     Matthew Woodring Stover  Science fiction   
489             Joseph O'Connor           Comedy   
502                   Anne Rice           Horror   
1101  John Ronald Reuel Tolkien          Fantasy   
1446         Robert A. Heinlein  Science fiction   
1450              William Arrow  Science fiction   
2861               Ken Grimwood          Fantasy   
2876        Clive Staples Lewis          Fantasy   
3305          Diana Wynne Jones          Fantasy   
3306          Madeleine L'Engle  Science fiction   
3582                Jules Verne  Science fiction   
4508               Terry Brooks          Fantasy   
5011            R. Scott Bakker          Fantasy   
5072                 Iain Banks  Science fiction   
5089               Isaac Asimov  Science fiction   

                                              thumbnail  \
24    http://books.google.com/books/content?id=KI66c...   
486   http://books.google.com/books/content?id=VbICO...   
489   http://books.google.com/books/content?id=DrE3I...   
502   http://books.google.com/books/content?id=cIn8T...   
1101  http://books.google.com/books/content?id=22ePu...   
1446  http://books.google.com/books/content?id=ZQhiP...   
1450                                                NaN   
2861  http://books.google.com/books/content?id=9vmNP...   
2876                                                NaN   
3305  http://books.google.com/books/content?id=hB7hA...   
3306                                                NaN   
3582                                                NaN   
4508  http://books.google.com/books/content?id=jqVsA...   
5011  http://books.google.com/books/content?id=BG8qG...   
5072                                                NaN   
5089  http://books.google.com/books/content?id=8ClIP...   

                                            description  published_year  \
24    Emma Watson a research physician has been trai...          2004.0   
486   From the depths of catastrophe, a glimmer of h...          2002.0   
489   In aid of Amnesty International, this is a bri...          2002.0   
502   Lestat Is Back, Saviour And Demon, Presiding O...          2003.0   
1101  Tolkien's Silmarillion is the core work of the...          1999.0   
1446  Epic, entertaining, Stranger in a Strange Land...          2005.0   
1450  The first novel in the series, "Return to the ...          1976.0   
2861  At forty-three Jeff Winston is tired of his lo...          2005.0   
2876  Four English school children enter the magic l...          1995.0   
3305  Howl's moving castle - Eldest of three sisters...         

In [28]:
books.to_csv("books_with_categories.csv", index=False)